In [18]:
import ee
import geemap
ee.Initialize()

ModuleNotFoundError: No module named 'ui'

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement ui (from versions: none)
ERROR: No matching distribution found for ui


In [77]:

def get_dates (img):
  doy = ee.Date(img.get('system:time_start')).getRelative('day', 'year')
  return img.set('doy', doy)

def median_vis(img):
  # Ensure that startDoy is a number.
  doyCol = ee.ImageCollection.fromImages(img.get('doy_matches'))
  # Filter images by date range; starting with the current startDate and
  # ending 15 days later. Reduce the resulting image collection by median.
  return doyCol.reduce(ee.Reducer.median())


#Only works for africa, need to adjust region geom to make bounding box around 
def make_visualization(wld_rgn):
    # Make a day-of-year sequence from 1 to 365 with a 16-day step.
  doyList = ee.List.sequence(1, 365, 16)

  # Import a MODIS NDVI collection.
  ndviCol = ee.ImageCollection('MODIS/006/MOD13A2').select('NDVI')

  Africa_map = geemap.Map()

  Africa_map.add_layer(ndviCol.first(), name = 'Raw NDVI')


  # Define a mask to clip the NDVI data by.
  mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('wld_rgn', wld_rgn))

  Africa_map.add_layer(mask, name = 'mask of Africa')


  # Define the regional bounds of animation frames.
  region = ee.Geometry.Polygon(
    [[[-18.698368046353494, 38.1446395611524],
      [-18.698368046353494, -36.16300755581617],
      [52.229366328646506, -36.16300755581617],
      [52.229366328646506, 38.1446395611524]]]
  )

  ndviCol = ndviCol.map(get_dates)

  distinctDOY = ndviCol.filterDate('2000-01-01', '2001-01-01')




  # Define a filter that identifies which images from the complete collection
  # match the DOY from the distinct DOY collection.
  filter = ee.Filter.equals(leftField = 'doy', rightField = 'doy')


  # Define a join.
  join = ee.Join.saveAll('doy_matches')


  # Apply the join and convert the resulting FeatureCollection to an
  # ImageCollection.
  joinCol = ee.ImageCollection(join.apply(distinctDOY, ndviCol, filter))

  Africa_map.add_layer(joinCol.first(), name = 'Unassembeled, Reasembled')


  # Map over the list of days to build a list of image composites, this builds a ImageCollection.
  ndviCompCol = joinCol.map(median_vis)

  Africa_map.add_layer(ndviCompCol.first(), name = 'median_vis Display')


  # Define RGB visualization parameters.
  visParams = {
    'bands' : ['B1', 'B2'],
    'min' : 0.0,
    'max' : 9000.0, 
    'palette' : ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
      '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
      '012E01', '011D01', '011301']
  }

  # Create RGB visualization images for use as animation frames.
  rgbVis = ndviCompCol.map(lambda img : img.visualize( min = 0.0, max = 9000.0, palette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
      '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
      '012E01', '011D01', '011301']).clip(mask))

  Africa_map.add_layer(rgbVis.first(), name = 'rgb NDVI Display')

  # Define GIF visualization parameters.
  gifParams = {
    'region': region,
    'dimensions': 600,
    'crs': 'EPSG:3857',
    'framesPerSecond': 10
  }

  # Print the GIF URL to the console.
  return rgbVis.getVideoThumbURL(gifParams)


make_visualization('Africa')



'https://earthengine.googleapis.com/v1/projects/earthengine-legacy/videoThumbnails/9cc4ad31de3503337f8a5705767c0124-64ea177287d5ef0565139c49b1a51332:getPixels'